In [ ]:
from transformers import pipeline
from datasets import load_dataset

In [3]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing TFBartForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForSequenceClassification for predictions without further training.


In [9]:
sequence_to_classify = "one day I will see the world"
candidate_labels = ['travel', 'cooking', 'dancing']
prediction = classifier(sequence_to_classify, candidate_labels)

# Get the predicted label with the highest score
predicted_label = prediction['labels'][0]

print(f"Sequence: {sequence_to_classify}")
print(f"Predicted category: {predicted_label}")

Sequence: one day I will see the world
Predicted category: travel


In [11]:
sequence_to_classify = "CS 614 assignment is fun to work on"
candidate_labels = ["positive", "negative", "neutral"]

# Use the classifier to predict the sentiment
sentiment_prediction = classifier(sequence_to_classify, candidate_labels)

# Print the predicted sentiment and its confidence score
print(f"Predicted sentiment: {sentiment_prediction['labels'][0]}")
print(f"Sentiment confidence score: {sentiment_prediction['scores'][0]}")

Predicted sentiment: positive
Sentiment confidence score: 0.9738269448280334


In [11]:
# Load the dataset
dataset = load_dataset('eastwind/semeval-2016-absa-reviews-english-translated-stanford-alpaca')

# Initialize the zero-shot classifier
classifier = pipeline("zero-shot-classification")

# Define the candidate labels for sentiment and category
sentiment_labels = ["positive", "negative", "neutral"]
category_labels = ["food", "service", "internet","hotel","rooms","tratment"] 

# Predicitng first 10 sequences in the dataset
for sequence in dataset['train']['sentence'][:10]:
    sentiment_prediction = classifier(sequence, sentiment_labels)
    category_prediction = classifier(sequence, category_labels)
    print(f"Sequence: {sequence}")
    print(f"Predicted sentiment: {sentiment_prediction['labels'][0]}")
    print(f"Sentiment confidence score: {sentiment_prediction['scores'][0]}")
    print(f"Predicted category: {category_prediction['labels'][0]}")
    print(f"Category confidence score: {category_prediction['scores'][0]}")

No model was supplied, defaulted to FacebookAI/roberta-large-mnli and revision 130fb28 (https://huggingface.co/FacebookAI/roberta-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFRobertaForSequenceClassification.

All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Sequence: great service my family and i visited the porto sokhna hotel where the wonderful service and quick response from all the workers where we booked 3 rooms on the lake and the reception section took us to the sea without additional fees and we thanked mr. mohammed for the manager of the front offices and in general the food was good and the service is excellent from all the workers but the beach needs more expansion and attention.
Predicted sentiment: positive
Sentiment confidence score: 0.9462096691131592
Predicted category: service
Category confidence score: 0.349272221326828
Sequence: great service my family and i visited the porto sokhna hotel where the wonderful service and quick response from all the workers where we booked 3 rooms on the lake and the reception section took us to the sea without additional fees and we thanked mr. mohammed for the manager of the front offices and in general the food was good and the service is excellent from all the workers but the beach ne

## Single Label

In [5]:
# Load the dataset
dataset = load_dataset('eastwind/semeval-2016-absa-reviews-english-translated-stanford-alpaca')

# Initialize the zero-shot classifier
classifier = pipeline("zero-shot-classification")

# Define the candidate labels for sentiment and category
sentiment_labels = ["positive", "negative", "neutral"]
# Get the unique categories
category_labels = list(set(dataset['train']['category']))

# Initialize counters for the total number of examples and the number of correct predictions
total = 0
correct_sentiment = 0
correct_category = 0

# Use the model to predict the sentiment and category of the sequences in the dataset
for sequence, true_sentiment, true_category in zip(dataset['train']['sentence'][:10], dataset['train']['sentiment'][:10], dataset['train']['category'][:10]):
    sentiment_prediction = classifier(sequence, sentiment_labels)
    category_prediction = classifier(sequence, category_labels)

    print(f"Sequence: {sequence}")
    print(f"Predicted sentiment: {sentiment_prediction['labels'][0]}")
    print(f"Actual sentiment: {true_sentiment}")
    print(f"Sentiment confidence score: {sentiment_prediction['scores'][0]}")
    print(f"Predicted category: {category_prediction['labels'][0]}")
    print(f"Actual category: {true_category}")
    print(f"Category confidence score: {category_prediction['scores'][0]}")
    
    # Check if the predictions are correct
    if sentiment_prediction['labels'][0] == true_sentiment:
        correct_sentiment += 1
    if category_prediction['labels'][0] == true_category:
        correct_category += 1
    total += 1

# Calculate the accuracy for sentiment and category
sentiment_accuracy = correct_sentiment / total
category_accuracy = correct_category / total

print(f"Sentiment accuracy: {sentiment_accuracy}")
print(f"Category accuracy: {category_accuracy}")

No model was supplied, defaulted to FacebookAI/roberta-large-mnli and revision 130fb28 (https://huggingface.co/FacebookAI/roberta-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFRobertaForSequenceClassification.

All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Sequence: great service my family and i visited the porto sokhna hotel where the wonderful service and quick response from all the workers where we booked 3 rooms on the lake and the reception section took us to the sea without additional fees and we thanked mr. mohammed for the manager of the front offices and in general the food was good and the service is excellent from all the workers but the beach needs more expansion and attention.
Predicted sentiment: positive
Actual sentiment: negative
Sentiment confidence score: 0.9462096691131592
Predicted category: HOTEL#QUALITY
Actual category: FOOD_DRINKS#QUALITY
Category confidence score: 0.07453253120183945
Sequence: great service my family and i visited the porto sokhna hotel where the wonderful service and quick response from all the workers where we booked 3 rooms on the lake and the reception section took us to the sea without additional fees and we thanked mr. mohammed for the manager of the front offices and in general the food was

## Multi Label

Given the model can predict more than one category in the given the dataset, it makes sense to turn the multi label prediction.
The single prediction accuracy is not great given the predicted label is only one of the actual labels and is not the same the actual one

In [7]:
# Load the dataset
dataset = load_dataset('eastwind/semeval-2016-absa-reviews-english-translated-stanford-alpaca')

# Initialize the zero-shot classifier
classifier = pipeline("zero-shot-classification")

# Define the candidate labels for sentiment and category
sentiment_labels = ["positive", "negative", "neutral"]
# Get the unique categories
category_labels = list(set(dataset['train']['category']))

# Combine the sentiment and category labels into a single list of candidate labels
candidate_labels = sentiment_labels + category_labels

# Initialize counters for the total number of examples and the number of correct predictions
total = 0
correct = 0

# Use the model to predict the sentiment and category of the sequences in the dataset
for sequence, true_sentiment, true_category in zip(dataset['train']['sentence'][:10], dataset['train']['sentiment'][:10], dataset['train']['category'][:10]):
    prediction = classifier(sequence, candidate_labels)
    
    # Get the predicted labels
    predicted_labels = [label for label, score in zip(prediction['labels'], prediction['scores']) ]

    print(f"Sequence: {sequence}")
    print(f"Predicted labels: {predicted_labels}")
    print(f"Actual sentiment: {true_sentiment}")
    print(f"Actual category: {true_category}")
    print(f"Sentiment confidence score: {sentiment_prediction['scores'][0]}")
    print(f"Category confidence score: {category_prediction['scores'][0]}")

    # Check if the predicted labels include the true sentiment and category
    if true_sentiment in predicted_labels and true_category in predicted_labels:
        correct += 1
    total += 1

# Calculate the accuracy
accuracy = correct / total

print(f"Accuracy: {accuracy}")

No model was supplied, defaulted to FacebookAI/roberta-large-mnli and revision 130fb28 (https://huggingface.co/FacebookAI/roberta-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFRobertaForSequenceClassification.

All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Sequence: great service my family and i visited the porto sokhna hotel where the wonderful service and quick response from all the workers where we booked 3 rooms on the lake and the reception section took us to the sea without additional fees and we thanked mr. mohammed for the manager of the front offices and in general the food was good and the service is excellent from all the workers but the beach needs more expansion and attention.
Predicted labels: ['positive', 'HOTEL#QUALITY', 'SERVICE#GENERAL', 'ROOMS#QUALITY', 'ROOMS_AMENITIES#QUALITY', 'FACILITIES#COMFORT', 'LOCATION#GENERAL', 'FACILITIES#QUALITY', 'ROOMS_AMENITIES#GENERAL', 'FACILITIES#GENERAL', 'ROOMS_AMENITIES#COMFORT', 'ROOMS#COMFORT', 'HOTEL#GENERAL', 'HOTEL#COMFORT', 'HOTEL#DESIGN_FEATURES', 'ROOMS#GENERAL', 'ROOMS_AMENITIES#PRICES', 'ROOMS_AMENITIES#CLEANLINESS', 'ROOMS#PRICES', 'neutral', 'FACILITIES#PRICES', 'FOOD_DRINKS#QUALITY', 'ROOMS_AMENITIES#DESIGN_FEATURES', 'ROOMS#DESIGN_FEATURES', 'HOTEL#PRICES', 'FOOD_DRIN